# Build YOLOv7 App Pipeline
A script that build a YOLOv7 app to run inference at the edge.

To the left in the file browser pane, you can find the /cloudstorage folder. Under /cloudstorage/models we store different versions of the machine learning models in version-named folders.

When a new version of the sustayn model is created, put it in `/cloudstorage/models/<version>/<all the files>` and update the `model_name` argument below in the Define the pipeline section.

When that is done, run this notebook.
    
## Install prerequisites

In [1]:
# Install the SDK
# !pip3 uninstall kfp
!pip3 install kfp==1.8.22 --user

# kfp-k8s for kfp v2
# !pip3 install pip install kfp-kubernetes --user

In [1]:
import kfp
from kfp import compiler, dsl, components
from kfp.components import load_component_from_text, InputPath, InputTextFile, OutputPath, OutputTextFile
from kfp.components import func_to_container_op

from typing import NamedTuple

print(kfp.__version__)

@func_to_container_op
def get_metadata_args_op(
    image_name: str,
    model_name: str,
    domain: str,
    namespace: str) -> NamedTuple(
  'PipelineArgs',
  [
    ('project', str),
    ('namespace', str),
    ('registry', str),
    ('bucket_name', str),
    ('image_name', str)
  ]):
    print("Preparing metadata for build {}".format(model_name))
    import urllib.request
    from collections import namedtuple
    
    def getMetadataValue(url):
        req = urllib.request.Request(url)
        req.add_header("Metadata-Flavor", "Google")
        return urllib.request.urlopen(req).read().decode()
    
    url = "http://metadata.google.internal/computeMetadata/v1/project/project-id"
    project_id = getMetadataValue(url)
    print("Project ID {}".format(project_id))

    output = namedtuple('PipelineArgs', ['project', 'namespace', 'registry', 'bucket_name', 'image_name'])
    return output(
        project_id, 
        namespace, 
        f'us-central1-docker.pkg.dev/{project_id}/{namespace}', 
        f'{namespace}.{domain}',
        f'us-central1-docker.pkg.dev/{project_id}/{namespace}/{image_name}'
    )


def prepare_model_files(
    model_name: str,
    bucket_name: str,
    weightsfile_path: OutputPath(str),
    namesfile_path: OutputPath(str)):
    '''Prepares Model files'''
    print("Preparing model files for {}".format(model_name))
    
    import os
    from google.cloud import storage
    from collections import namedtuple

    weights_file = 'best_weights.pt'
    names_file = 'object.names'

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    
    def download_blob(source_blob_name, destination_file_name):
        """Downloads a blob from the bucket."""
        print("Downloading object {} to {}.".format(source_blob_name, destination_file_name))
        blob = bucket.blob(source_blob_name)
        blob.download_to_filename(destination_file_name)

    weights_blob = f'models/{model_name}/{weights_file}'
    download_blob(weights_blob, weightsfile_path)
    
    names_blob =f'models/{model_name}/{names_file}'
    download_blob(names_blob, namesfile_path)
    

prepare_model_files_op = components.create_component_from_func(
    prepare_model_files,
    packages_to_install=['google-cloud-storage'],
    output_component_file='yolov7_prepare_model_files.yaml')


def prepare_dockerfile(
    model_name: str,
    img_size: int,
    dockerfile_path: OutputPath(str)):
    '''Prepares Dockerfiles and Manifests'''
    print("Preparing docker file for {}".format(model_name))
    
    import os
    from collections import namedtuple
    import requests
    
    def wget(url, file):
        r = requests.get(url, allow_redirects=True)
        print(f'Downloading {url} to {file}')
        open(file, 'wb').write(r.content)
    
    def write_file(file_path, content):
        with open(file_path, 'w') as f:
            print(content)
            f.write(content)
    
    # Remember that we still cannot do multi-arch builds in kubeflow
    # All commands in the Dockerfile must be multi-arch safe
    docker_file_template = f"""
ARG BASE_IMAGE
ARG BASE_IMAGE_TAG
FROM $BASE_IMAGE:$BASE_IMAGE_TAG

ENV MODEL_NAME={model_name}
ENV TRAINING_DATASET={model_name}
ENV IMG_SIZE={img_size}

ADD best_weights.pt /usr/src/app/model.pt
ADD object.names /usr/src/app/object.names
"""
    write_file(dockerfile_path, docker_file_template)
    

prepare_dockerfile_op = components.create_component_from_func(
    prepare_dockerfile,
    packages_to_install=['chevron', 'requests'],
    output_component_file='ppe_prepare_dockerfile.yaml')


1.8.22


## Define the pipeline

In [2]:
# import os
import kfp
from kfp import compiler, dsl, components
from typing import NamedTuple
from kubernetes.client.models import V1EnvVar, V1Volume, V1VolumeMount, V1SecurityContext

pipeline_name = 'Build YOLOv7 App'
pipeline_description = 'A pipeline to build custom YOLOv7 Apps'

@dsl.pipeline(
name = pipeline_name,
description = pipeline_description)
def build_yolov7_app(
    image_name: str,
    model_name: str,
    img_size: int,
    namespace: str,
    domain: str):
    """A pipeline to build custom multi architecture YOLOv7 Apps"""
    
    get_metadata_args = get_metadata_args_op(
        image_name=image_name,
        model_name=model_name,
        namespace=namespace,
        domain=domain
    )
    get_metadata_args.execution_options.caching_strategy.max_cache_staleness = "P0D"

    prepare_model_files = prepare_model_files_op(
        model_name=model_name,
        bucket_name=get_metadata_args.outputs["bucket_name"]
    ).after(get_metadata_args)
    prepare_model_files.execution_options.caching_strategy.max_cache_staleness = "P0D"

    prepare_docker_file = prepare_dockerfile_op(
        model_name=model_name,
        img_size=img_size
    ).after(prepare_model_files)
    prepare_docker_file.execution_options.caching_strategy.max_cache_staleness = "P0D"

    # l4tr32.7.1
    # l4tr32.7.2
    # l4tr34.1.1
    # amd64
    # arm64
    # nv-amd64
    # nv-arm64
    
    def kaniko_executor(base_image, base_image_tag, dockerfile_path: InputPath(str), weights_file: InputPath(str), names_file: InputPath(str), image_name, model_name):
        return dsl.ContainerOp(
            name="Build and push app image",
            image="gcr.io/kaniko-project/executor:v1.6.0",
            command=["/kaniko/executor"],
            arguments=[
                "--build-arg", f"BASE_IMAGE={base_image}",
                "--build-arg", f"BASE_IMAGE_TAG={base_image_tag}",
                "--dockerfile", dockerfile_path,
                "--destination", f'{image_name}:l4tr34.1.1-{model_name}',
                "--label", "app_name=yolov7",
                "--label", "gpu=nvidia_l4t",
                "--label", "l4t_version=l4tr34.1.1"
            ],
            artifact_argument_paths=[weights_file, names_file]
        )
    
    kaniko_executor_l4tr32_7_1 = kaniko_executor(
        "gcr.io/teknoir/yolov7",
        "l4tr32.7.1",
        kfp.dsl.InputArgumentPath(argument=prepare_docker_file.outputs['dockerfile'], path='/workspace/Dockerfile'),
        kfp.dsl.InputArgumentPath(argument=prepare_model_files.outputs['weightsfile'], path='/workspace/best_weights.pt'),
        kfp.dsl.InputArgumentPath(argument=prepare_model_files.outputs['namesfile'], path='/workspace/object.names'),
        get_metadata_args.outputs["image_name"],
        model_name
    ).after(prepare_docker_file)
        
    kaniko_executor_l4tr32_7_2 = kaniko_executor(
        "gcr.io/teknoir/yolov7",
        "l4tr32.7.2",
        kfp.dsl.InputArgumentPath(argument=prepare_docker_file.outputs['dockerfile'], path='/workspace/Dockerfile'),
        kfp.dsl.InputArgumentPath(argument=prepare_model_files.outputs['weightsfile'], path='/workspace/best_weights.pt'),
        kfp.dsl.InputArgumentPath(argument=prepare_model_files.outputs['namesfile'], path='/workspace/object.names'),
        get_metadata_args.outputs["image_name"],
        model_name
    ).after(prepare_docker_file)
        
    kaniko_executor_l4tr34_1_1 = kaniko_executor(
        "gcr.io/teknoir/yolov7",
        "l4tr34.1.1",
        kfp.dsl.InputArgumentPath(argument=prepare_docker_file.outputs['dockerfile'], path='/workspace/Dockerfile'),
        kfp.dsl.InputArgumentPath(argument=prepare_model_files.outputs['weightsfile'], path='/workspace/best_weights.pt'),
        kfp.dsl.InputArgumentPath(argument=prepare_model_files.outputs['namesfile'], path='/workspace/object.names'),
        get_metadata_args.outputs["image_name"],
        model_name
    ).after(prepare_docker_file)

        

## Compile & upload pipeline

In [3]:
import uuid
import json

pipeline_version_file = pipeline_file = 'build_yolov7_app.yaml'
# 1h timeout
pipeline_conf = kfp.dsl.PipelineConf().set_timeout(3600*1).set_image_pull_policy(policy="Always")
workflow = kfp.compiler.Compiler().compile(pipeline_func=build_yolov7_app, 
                                           package_path=pipeline_file,
                                           pipeline_conf=pipeline_conf)
client = kfp.Client(namespace='teknoir')
filter = json.dumps({'predicates': [{'key': 'name', 'op': 1, 'string_value': pipeline_name}]})
pipelines = client.pipelines.list_pipelines(filter=filter)

if not pipelines.pipelines:
    pipeline = client.pipeline_uploads.upload_pipeline(pipeline_file, name=pipeline_name, description=pipeline_description)
else:
    pipeline_version_name = pipeline_name + f' - {str(uuid.uuid4())[:6]}'
    pipeline_version = client.pipeline_uploads.upload_pipeline_version(pipeline_version_file,
                                                                       name=pipeline_version_name,
                                                                       pipelineid=pipelines.pipelines[0].id)

/home/jovyan/.local/lib/python3.10/site-packages/kfp/dsl/_container_op.py:1261: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  warnings.warn(
ERROR:root:Failed to read a token from file '/var/run/secrets/kubeflow/pipelines/token' ([Errno 2] No such file or directory: '/var/run/secrets/kubeflow/pipelines/token').


## To run a pipeline
Now you can find the new pipeline, or version, in the menu to the left, under Workflow -> Pipelines.

### Run the pipeline from the Devstudio

There is a node, under `pipelines`, that allows you to run the pipeline.
In the node properties, the new pipeline is now listed in the pipeline dropdown.
The arguments are listed when you select pipeline, and there is an option to input default values.

### Pipeline arguments
Arguments example:

```
arguments = {
    'image_name': 'yolov7-ppe-hhsv',
    'model_name': 'ppe-bbox-multiclass-20220907231701674',
    'img_size': 416,
    'namespace': os.getenv('NAMESPACE', 'teknoir-ai'),
    'domain': os.getenv('DOMAIN', 'teknoir.cloud')
}
```

In [5]:
import os
domain = os.getenv('DOMAIN', 'teknoir.cloud')
namespace = os.getenv('NAMESPACE', 'teknoir-ai')

arguments = {
    'image_name': 'yolov7-ppe-hhsv',
    'model_name': 'ppe-bbox-multiclass-20220907231701674',
    'img_size': 416,
    'namespace': namespace,
    'domain': domain,
}

client = kfp.Client(namespace='teknoir')
run_result = client.create_run_from_pipeline_func(build_yolov7_app, arguments=arguments)

ERROR:root:Failed to read a token from file '/var/run/secrets/kubeflow/pipelines/token' ([Errno 2] No such file or directory: '/var/run/secrets/kubeflow/pipelines/token').


ApiException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'date': 'Tue, 03 Oct 2023 15:20:27 GMT', 'content-length': '1399', 'x-envoy-upstream-service-time': '8', 'server': 'envoy'})
HTTP response body: {"error":"Failed to authorize with API resource references: Failed to authorize with API resource references: PermissionDenied: User 'anders.aslund@teknoir.ai' is not authorized with reason:  (request: \u0026ResourceAttributes{Namespace:teknoir,Verb:list,Group:pipelines.kubeflow.org,Version:v1beta1,Resource:experiments,Subresource:,Name:,}): Unauthorized access","code":7,"message":"Failed to authorize with API resource references: Failed to authorize with API resource references: PermissionDenied: User 'anders.aslund@teknoir.ai' is not authorized with reason:  (request: \u0026ResourceAttributes{Namespace:teknoir,Verb:list,Group:pipelines.kubeflow.org,Version:v1beta1,Resource:experiments,Subresource:,Name:,}): Unauthorized access","d